In [75]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.dates import  DateFormatter
# %matplotlib inline
import fitsio
from fitsio import FITS, FITSHDR
import datetime
from astropy import units as u
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
import time

In [2]:
from some_func import cart2spheric, spheric2cart, spheric_coordinates_rotate, hor2pix_s1c, bright_relay_s1c

In [3]:
# Координаты расположения S1C
lat_s1c=56.1501667 # ШИРОТА
lon_s1c=46.1050833 # ДОЛГОТА
h_s1c=183.0
S1C_site=EarthLocation(lat=lat_s1c*u.deg, lon=lon_s1c*u.deg, height=h_s1c*u.m)

# Ориентация S1C - 26 августа
az0_s1c = 3.273747186726014;
alt0_s1c = 1.5470500778432814;
a_s1c = (0.17638446258114976, -0.0012472703206068809, -5.316537749985197e-5)
b_s1c = (0.19928459764658202, 5.100993511668623e-5, -0.0012452931691676276)
c_s1c = (134.36032720169513, -800.3341162043586, 34.16915787711284)
d_s1c = ( 165.53392397449926, -32.78284210934066, -801.5998710431686)

In [4]:
dark_filename='/mnt/Work_disk/Owncloud/0000_Two_points/DATA/01_S1C/140826/dark/s1c_aug26_dark_median.fit'
dark=fitsio.read(dark_filename)
dark.astype('d')
BS6300_catalog_filename='Haritonov_6300BS_catalog.npz'

In [5]:
npzfile=np.load(BS6300_catalog_filename)
CAT_FIELDS=npzfile['CAT_FIELDS']
CAT_6300BS=npzfile['CAT_6300BS']
CAT_SPTYPE=npzfile['CAT_SPTYPE']
print(CAT_FIELDS)

['HARITONOV_ID' 'BS_ID' 'RA (icrs, deg)' 'DEC (icrs, deg)' 'MAG_V'
 '6300 Flux (erg/(sm^2 * s *  sm))']


In [81]:
img_num_start=1
# img_num_end=1
img_num_end=745
img_num_list=[]
R_median_list=[]
R_median_sum=0.
time_axe=[]
ind=0
for img_num in range(img_num_start,img_num_end+1):
#     pic_save_name='/home/ashindin/temp/' + "{0:0>3}".format(img_num) + '.png'
    ind+=1
    pic_save_name='/home/ashindin/temp/s1c_aug26/' + "{0:0>4}".format(ind) + '.jpg'
    fig=plt.figure(figsize=(9,4.5))
    ax = plt.subplot(121)
    ax.grid(True)
    ax.set_xlim((100, 5000))
    ax.set_ylim((0, 5))
    plt.ylabel('Calibration coef. [R/ADCu]')
    plt.xlabel('Star brightness per pixel [R]')
    ax2=plt.subplot(122)
    ax2.set_xlim((1,288))
    ax2.set_ylim((288,1))
        
    fit_filename='/mnt/Work_disk/Owncloud/0000_Two_points/DATA/01_S1C/140826/aug26red15bin2_0' + "{0:0>3}".format(img_num) + '.fit'

    fitfile=fitsio.FITS(fit_filename)
    img=fitfile[0].read()
    img=img.astype('d')-dark
    plt.axes(ax2)
    plt.pcolormesh(img, cmap="gray", vmin=np.median(img)-100, vmax=np.median(img)+100)
    header=fitfile[0].read_header()
    fitfile.close()
    date_obs_str=header['COMMENT'][7:31]
    if date_obs_str[16]==' ':
        date_obs_str=date_obs_str[0:16] + '0' + date_obs_str[17:24]
    if date_obs_str[17]==' ':
        date_obs_str=date_obs_str[0:17] + '0' + date_obs_str[18:24]

    date_obs=datetime.datetime.strptime(date_obs_str,"%b %d %H:%M:%S.%f %Y") - datetime.timedelta(seconds=7.5) - datetime.timedelta(hours=4)
    plt.title('fr'+"{0:0>4}".format(img_num) + ' ' + str(date_obs)[0:23])
    time_axe.append(date_obs)
    
    print("{0:0>3}".format(img_num) + " "+ str(date_obs))

    BS_coord=SkyCoord(CAT_6300BS[:,2], CAT_6300BS[:,3], frame='icrs', unit='deg');
    
    altaz=BS_coord.transform_to(AltAz(obstime=date_obs, location=S1C_site,temperature=15*u.deg_C,pressure=1013*u.hPa,
                                       relative_humidity=0.5,obswl=630.0*u.nm))
    BS_AzAlt=np.zeros((np.size(CAT_6300BS,0),2))
    BS_AzAlt[:,0]=altaz.az.degree
    BS_AzAlt[:,1]=altaz.alt.degree
#     print(BS_AzAlt[28,0],BS_AzAlt[28,1])

    BS_xy=np.zeros((np.size(CAT_6300BS,0),2))
    BS_xy[:,0], BS_xy[:,1] = hor2pix_s1c (BS_AzAlt[:,0]*np.pi/180,BS_AzAlt[:,1]*np.pi/180, az0_s1c, alt0_s1c, c_s1c, d_s1c)

    # Catalog filtration
    filt_mask=np.zeros(np.size(CAT_6300BS,0),dtype=bool)
    for i in range(0,np.size(CAT_6300BS,0)):
        if BS_AzAlt[i,1]>=85.0:
            if BS_xy[i,0]>=1 and BS_xy[i,0]<=288:
                if BS_xy[i,1]>=1 and BS_xy[i,1]<=288:
                    filt_mask[i]=True
    BS_AzAlt_filt=BS_AzAlt[filt_mask,:]
    BS_xy_filt=BS_xy[filt_mask,:]
    CAT_6300BS_filt=CAT_6300BS[filt_mask,:]
    CAT_SPTYPE_filt=CAT_SPTYPE[filt_mask]
    if len(CAT_6300BS_filt)!=0:
        img_red_st_int=np.zeros(np.size(BS_xy_filt,0))
        img_red_st_num=np.zeros(np.size(BS_xy_filt,0))
        BS_adc_filt=np.zeros(np.size(BS_xy_filt,0))
        area_rad=4
        for j in range(np.size(BS_xy_filt,0)):    
            st_x=BS_xy_filt[j,0]
            st_y=BS_xy_filt[j,1]
            area=img[int(st_y)-area_rad:int(st_y)+area_rad+1, int(st_x)-area_rad:int(st_x)+area_rad+1]

            sum_temp=0.0
            num=0
            med=np.median(area)
            for i in range(len(area.flat)):
                if area.flat[i]>=1.3*med:
                    num+=1
                    sum_temp+=area.flat[i]-med
            img_red_st_int[j]=sum_temp
            img_red_st_num[j]=num
            if num>0:
                BS_adc_filt[j]=sum_temp/num

        BS_relay=np.zeros(np.size(BS_xy_filt,0))
        R_adc_coef=np.zeros(np.size(BS_xy_filt,0))
        for i in range(np.size(BS_xy_filt,0)):
             if img_red_st_num[i]>3:
                    BS_relay[i]=bright_relay_s1c(CAT_6300BS_filt[i,5],img_red_st_num[i])
                    if BS_adc_filt[i]>0 and BS_relay[i]>100:
                        R_adc_coef[i]=BS_relay[i]/BS_adc_filt[i]

    #     print(R_adc_coef[R_adc_coef.nonzero()])
        R_median=np.median(R_adc_coef[R_adc_coef.nonzero()])
        R_median_sum+=R_median

        print(R_median)
        R_median_list.append(R_median)
        img_num_list.append(img_num)
        
        plt.axes(ax)
        plt.plot([0, 5000], [R_median, R_median], c='r', lw=2)
        plt.scatter(BS_relay,R_adc_coef,s=np.pi*(img_red_st_num)**2)
        plt.ylabel('Calibration coef. [R/ADCu]')
        plt.xlabel('Star brightness per pixel [R]')

        plt.title(R_median)

        plt.axes(ax2)
        plt.plot(BS_xy_filt[:,0],BS_xy_filt[:,1],color="r",marker=".", lw=0.,mec="r", mfc="r")
        
        for ii in range(len(CAT_6300BS_filt)):
            ax2.text(BS_xy_filt[ii,0]-10, BS_xy_filt[ii,1]-10, str(int(CAT_6300BS_filt[ii,0])), ha="center", va="center", size=12)
        
        
        # plt.show()
    plt.savefig(pic_save_name)
    plt.close()
print(' ')
print(R_median_sum/(img_num_end-img_num_start+1))

001 2014-08-26 18:11:38.147000
3.15480481699
002 2014-08-26 18:11:53.147000
2.95117142018
003 2014-08-26 18:12:08.163000
2.80013994852
004 2014-08-26 18:12:23.163000
2.72539164918
005 2014-08-26 18:12:38.163000
2.76545742221
006 2014-08-26 18:12:53.163000
2.77008533802
007 2014-08-26 18:13:08.179000
2.69340985492
008 2014-08-26 18:13:23.179000
2.69777928198
009 2014-08-26 18:13:38.179000
2.68786958264
010 2014-08-26 18:13:53.179000
2.70004925234
011 2014-08-26 18:14:08.179000
2.97541745358
012 2014-08-26 18:14:23.194000
3.29213812397
013 2014-08-26 18:14:38.194000
3.45001136849
014 2014-08-26 18:14:53.194000
3.54068903543
015 2014-08-26 18:15:08.194000
3.87555360737
016 2014-08-26 18:15:23.210000
4.51175848081
017 2014-08-26 18:15:38.210000
4.54429842469
018 2014-08-26 18:16:10.350000
4.17052698714
019 2014-08-26 18:16:25.366000
3.94412154507
020 2014-08-26 18:16:40.366000
3.71651122507
021 2014-08-26 18:16:55.366000
3.63417525116
022 2014-08-26 18:17:10.366000
3.37943453972
023 2014-0

/usr/local/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/usr/local/lib/python3.5/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


081 2014-08-26 18:31:55.558000
nan
082 2014-08-26 18:32:10.558000
nan
083 2014-08-26 18:32:25.558000
nan
084 2014-08-26 18:32:40.558000
nan
085 2014-08-26 18:32:55.574000
nan
086 2014-08-26 18:33:10.574000
nan
087 2014-08-26 18:33:25.574000
13.5843898084
088 2014-08-26 18:33:40.574000
10.6879044582
089 2014-08-26 18:33:55.574000
8.82306640131
090 2014-08-26 18:34:10.589000
10.7278592412
091 2014-08-26 18:34:25.589000
5.24146547402
092 2014-08-26 18:34:40.589000
7.1832349112
093 2014-08-26 18:34:55.589000
6.00670297651
094 2014-08-26 18:35:10.605000
7.46832100722
095 2014-08-26 18:35:25.605000
5.95065287097
096 2014-08-26 18:35:40.605000
4.35463178608
097 2014-08-26 18:35:55.605000
4.80888537415
098 2014-08-26 18:36:10.605000
5.41453273024
099 2014-08-26 18:36:25.620000
5.11533395192
100 2014-08-26 18:36:40.620000
4.45433037955
101 2014-08-26 18:36:55.620000
3.88848556508
102 2014-08-26 18:37:10.620000
3.58376815114
103 2014-08-26 18:37:25.636000
3.31949375017
104 2014-08-26 18:37:40.63

In [48]:
dates = matplotlib.dates.date2num(time_axe)
dates2=dates[np.array(img_num_list)-1]

In [84]:
fig=plt.figure(figsize=(9,8))
ax = plt.subplot(211)
# plt.scatter(img_num_list,R_median_list)
plt.plot_date(dates2,R_median_list)
#ax.set_ylim((0,5))
ax.grid(True)
# plt.gcf().autofmt_xdate()
ax.xaxis.set_major_formatter( DateFormatter('%H:%M') )
plt.ylabel('Calibration coef. [R/ADCu]')

plt.title('S1C (26 aug 2014) calibration curve')

ax2=plt.subplot(212)
plt.plot_date(dates2,R_median_list)
ax2.set_ylim((0,5))
ax2.grid(True)
# plt.gcf().autofmt_xdate()
ax2.xaxis.set_major_formatter( DateFormatter('%H:%M') )
plt.ylabel('Calibration coef. [R/ADCu]')

# plt.show()
plt.savefig('/home/ashindin/temp/s1c_aug26.png')

In [9]:
np.std(R_median_list[550:683])

0.074451234548637266

In [10]:
np.median(R_median_list[550:683])

2.3258883366173717

In [11]:
len(R_median_list)

686

In [12]:
np.mean(R_median_list[420:687])

2.2416773796103246

In [13]:
RR_median_list=np.array(R_median_list)

In [14]:
RR_median_list[(RR_median_list>0).nonzero()]

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:1: RuntimeWarning: invalid value encountered in greater
  if __name__ == '__main__':


array([  3.15480482,   2.95117142,   2.80013995,   2.72539165,
         2.76545742,   2.77008534,   2.69340985,   2.69777928,
         2.68786958,   2.70004925,   2.97541745,   3.29213812,
         3.45001137,   3.54068904,   3.87555361,   4.51175848,
         4.54429842,   4.17052699,   3.94412155,   3.71651123,
         3.63417525,   3.37943454,   3.63889887,   3.38807833,
         3.74855932,   4.23106369,   4.84366944,   5.29056304,
         5.14538994,   4.89684463,   5.68989437,   5.60205586,
         4.84600824,   5.57095924,   5.78710357,   5.59668912,
         5.33165899,   5.05278492,   4.96099015,   5.15199336,
         4.36256599,   4.17052699,   5.39417734,   5.08812473,
         4.38122496,   4.83929569,   5.14975746,   5.33898111,
         5.05897285,   5.1085044 ,   4.85156779,   4.37454626,
         3.7426832 ,   4.03189652,   4.18476463,   4.27834864,
         4.39801126,   4.33326138,   3.92169778,   4.32509773,
         5.17063486,   5.70233949,   5.40433587,   5.70

In [15]:
np.median(RR_median_list[(RR_median_list>0).nonzero()])

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:1: RuntimeWarning: invalid value encountered in greater
  if __name__ == '__main__':


2.5273717406037344

In [16]:
np.median(RR_median_list)

/usr/local/lib/python3.5/site-packages/numpy/lib/function_base.py:3569: RuntimeWarning: Invalid value encountered in median
  RuntimeWarning)


nan